In [1]:
#Import Header Files
import pandas as pd
from sklearn.model_selection import train_test_split 
import time
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
import pickle
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
import warnings

In [2]:
#Preprocessing data
def split_scalar(indep_X,dep_Y):
        X_train, X_test, y_train, y_test = train_test_split(indep_X, dep_Y, test_size = 0.25, random_state = 0)
        sc = StandardScaler()
        X_train = sc.fit_transform(X_train)
        X_test = sc.transform(X_test)    
        return X_train, X_test, y_train, y_test

In [3]:
#Find evaluation matrices
def r2_prediction(regressor,X_test,y_test):
     y_pred = regressor.predict(X_test)
     from sklearn.metrics import r2_score
     r2=r2_score(y_test,y_pred)
     return r2
 

In [4]:
#Linear Algorithm
def Linear(X_train,y_train,X_test):       
        # Fitting K-NN to the Training set
        from sklearn.linear_model import LinearRegression
        regressor = LinearRegression()
        regressor.fit(X_train, y_train)
        r2=r2_prediction(regressor,X_test,y_test)
        return  r2   
    

In [5]:
#SVM Linear
def svm_linear(X_train,y_train,X_test):        
        regressor = SVR(kernel = 'linear')
        regressor.fit(X_train, y_train)
        r2=r2_prediction(regressor,X_test,y_test)
        return  r2  
    

In [6]:
#SVM NoN Linear
def svm_NL(X_train,y_train,X_test):     
        regressor = SVR(kernel = 'rbf')
        regressor.fit(X_train, y_train)
        r2=r2_prediction(regressor,X_test,y_test)
        return  r2  
     


In [7]:
#Decision tree
def Decision(X_train,y_train,X_test):
        regressor = DecisionTreeRegressor(random_state = 0)
        regressor.fit(X_train, y_train)
        r2=r2_prediction(regressor,X_test,y_test)
        return  r2  
     

In [8]:
#ramdom Forest
def random(X_train,y_train,X_test):       
        # Fitting K-NN to the Training set
        from sklearn.ensemble import RandomForestRegressor
        regressor = RandomForestRegressor(n_estimators = 10, random_state = 0)
        regressor.fit(X_train, y_train)
        r2=r2_prediction(regressor,X_test,y_test)
        return  r2 
    

In [9]:
#Create RFE Function
def rfeFeature(indep_X,dep_Y,n):
        rfelist=[]
        lin = LinearRegression()      
        SVRl = SVR(kernel = 'linear')      
        #SVRnl = SVR(kernel = 'rbf')
        dec = DecisionTreeRegressor(random_state = 0)
        rf = RandomForestRegressor(n_estimators = 10, random_state = 0)
        rfemodellist=[lin,SVRl,dec,rf] 
        for i in   rfemodellist:
            print(i)
            log_rfe = RFE(estimator=i,n_features_to_select=n)
            log_fit = log_rfe.fit(indep_X, dep_Y)
            log_rfe_feature=log_fit.transform(indep_X)
            rfelist.append(log_rfe_feature)

        return rfelist

In [10]:
#Create output in table Format
def rfe_regression(acclin,accsvml,accsvmnl,accdes,accrf): 
    rfedataframe=pd.DataFrame(index=['Linear','SVRl','Random','DecisionTree'],columns=['Linear','SVMl',
                                                                                        'Decision','Random'])

    for number,idex in enumerate(rfedataframe.index):
        
        rfedataframe['Linear'][idex]=acclin[number]       
        rfedataframe['SVMl'][idex]=accsvml[number]
        #rfedataframe['SVMnl'][idex]=accsvmnl[number]
        rfedataframe['Decision'][idex]=accdes[number]
        rfedataframe['Random'][idex]=accrf[number]
    return rfedataframe


In [11]:
#Read Dataset
dataset1=pd.read_csv("prep.csv",index_col=None)
df2=dataset1
df2 = pd.get_dummies(df2, drop_first=True)

indep_X=df2.drop('classification_yes',axis=1)
dep_Y=df2['classification_yes']


In [12]:
#Apply Best Feature in Regression Algorithm
rfelist=rfeFeature(indep_X,dep_Y,5)       

acclin=[]
accsvml=[]
accsvmnl=[]
accdes=[]
accrf=[]


for i in rfelist:   
    X_train, X_test, y_train, y_test=split_scalar(i,dep_Y)  
    r2_lin=Linear(X_train,y_train,X_test)
    acclin.append(r2_lin)
    
    r2_sl=svm_linear(X_train,y_train,X_test)    
    accsvml.append(r2_sl)
    
    r2_NL=svm_NL(X_train,y_train,X_test)
    accsvmnl.append(r2_NL)
    
    r2_d=Decision(X_train,y_train,X_test)
    accdes.append(r2_d)
    
    r2_r=random(X_train,y_train,X_test)
    accrf.append(r2_r)
    

LinearRegression()
SVR(kernel='linear')
DecisionTreeRegressor(random_state=0)
RandomForestRegressor(n_estimators=10, random_state=0)


In [13]:
warnings.filterwarnings("ignore")

In [14]:
#Output
result=rfe_regression(acclin,accsvml,accsvmnl,accdes,accrf)
result


,Linear,SVMl,Decision,Random
Linear,0.620124,0.457136,0.77924,0.780135
SVRl,0.604508,0.456848,0.776474,0.776745
Random,0.674403,0.628206,0.696181,0.815538
DecisionTree,0.686361,0.64332,0.836806,0.845303
